Если count_over равен 'columns', верните среднее тензора по колонкам. Если равен 'rows', то верните среднее по рядам. Гарантируется, что тензор будет матрицей (то есть будет иметь размерность 2).

In [2]:
import torch

def function01(tensor: torch.Tensor, count_over: str) -> torch.Tensor:
    if count_over == 'columns':
        return tensor.mean(dim=0)
    elif count_over == 'rows':
        return tensor.mean(dim=1)

In [4]:
t = torch.tensor([[1.0, 2.0, 3.0], 
                  [4.0, 5.0, 6.0]])

print("Среднее по колонкам:", function01(t, 'columns'))
print("Среднее по рядам:", function01(t, 'rows'))

Среднее по колонкам: tensor([2.5000, 3.5000, 4.5000])
Среднее по рядам: tensor([2., 5.])


Напишите функцию function02. Функции на вход должен приходить датасет — тензор-матрица признаков объектов. Ваша функция должна создать тензор-вектор с весами (пусть они будут из равномерного распределения на отрезке от 0 до 1) и вернуть их для дальнейшего обучения линейной регрессии без свободного коэффициента. Сделайте эти веса типа float32, для них нужно будет в процессе обучения вычислять градиенты (воспользуйтесь requires_grad).

In [1]:
import torch

def function02(dataset: torch.Tensor) -> torch.Tensor:
    num_features = dataset.shape[1]
    weights = torch.rand(num_features, dtype=torch.float32,  requires_grad=True)
    return weights
    

In [2]:
dataset = torch.tensor([[1.0, 2.0], 
                        [3.0, 4.0], 
                        [5.0, 6.0]])

weights = function02(dataset)
    
print("Сгенерированные веса:", weights)
print("Тип весов:", weights.dtype)
print("Градиенты включены:", weights.requires_grad)

Сгенерированные веса: tensor([0.6150, 0.4470], requires_grad=True)
Тип весов: torch.float32
Градиенты включены: True


Напишите функцию function03. Она должна принимать тензор-матрицу с объектами и тензор-вектор с правильными ответами, будем решать задачу регрессии: def function03(x: torch.Tensor, y: torch.Tensor):

Создайте внутри функции веса для линейной регрессии (без свободного коэффициента), можете воспользоваться функцией из предыдущего степа. С помощью градиентного спуска подберите оптимальные веса для входных данных (используйте длину шага около 1e-2). Верните тензор-вектор с оптимальными весами из функции. Ваши обученные веса должны давать MSE на обучающей выборке меньше единицы.

In [8]:
import torch


def function02(dataset: torch.Tensor) -> torch.Tensor:
    num_features = dataset.shape[1]
    weights = torch.rand(num_features, dtype=torch.float32,  requires_grad=True)
    return weights



def function03(x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
    
    weights = function02(x)

    step = 1e-2
    max_epochs = 1000
    target_loss = 1.0
    
    for epoch in range(max_epochs):
        y_pred = x @ weights

        loss = torch.mean((y_pred - y)**2)

        if loss.item() < target_loss:
            break

        # Вычисляем градиенты 
        loss.backward()

        # Обновляем веса
        with torch.no_grad():  # Отключаем подсчет градиентов
            weights -= step * weights.grad  # Обновляем веса
            weights.grad.zero_()  # Обнуляем градиенты для следующей итерации

    return weights.detach()  # Возвращаем веса без градиентов
    

In [9]:
n_features = 2
n_objects = 300
w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2
trained_weights = function03(X, Y)
print("Trained Weights:", trained_weights)

Trained Weights: tensor([-0.1173,  0.5893])


Напишите функцию function04. Она должна принимать тензор-матрицу с объектами и тензор с правильными ответами, будем решать задачу регрессии: def function04(x: torch.Tensor, y: torch.Tensor):

Создайте внутри функции полносвязный слой, обучите этот полносвязный слой на входных данных с помощью градиентного спуска (используйте длину шага около 1e-2). Верните его из функции. Ваш полносвязный слой должен давать MSE на обучающей выборке меньше 0.3.

In [3]:
import torch.nn as nn
import torch


def function04(x: torch.Tensor, y: torch.Tensor) -> nn.Linear:

    layer = nn.Linear(in_features=x.shape[1], out_features=1, bias=True)
    
    max_epochs = 1000
    target_loss = 0.3
    step = 1e-2
    
    for epoch in range(max_epochs):
        
        y_pred = layer(x).ravel()
        
        loss = torch.mean((y_pred - y) ** 2)

        if loss < target_loss:
            break
            
        loss.backward()
    
        with torch.no_grad():
            layer.weight -= layer.weight.grad * step
            layer.bias -= layer.bias.grad * step

        layer.zero_grad()
        
    return layer

In [4]:
n_features = 2
n_objects = 300

# Генерация случайных данных
w_true = torch.randn(n_features)
X = (torch.rand(n_objects, n_features) - 0.5) * 5
Y = X @ w_true + torch.randn(n_objects) / 2

# Вызов функции
trained_layer = function04(X, Y)

# Пример предсказания
y_pred = trained_layer(X).squeeze()
print(y_pred)


tensor([ 0.2251,  1.3900, -0.0158,  0.3216, -1.1757,  1.1547, -1.6405, -0.0609,
        -0.5239, -0.3908, -1.6466, -1.0907, -1.3562, -0.1496,  0.0340, -0.6154,
        -0.4205,  0.0432, -0.0790,  0.2065,  0.2199,  1.0290,  0.4581, -1.1917,
        -1.2237, -1.2897,  0.7580,  1.0098, -0.8459, -0.6801, -0.9560,  0.2422,
        -1.6604, -1.0423,  0.3711, -0.8397, -0.0408,  0.5658, -1.6615, -2.0187,
        -1.6977, -1.6872,  0.4920, -0.1068, -1.4701, -0.7962,  0.2969,  0.5574,
        -1.1478, -0.6930, -1.3691, -0.1330, -1.3965, -0.7004,  0.6694, -2.0224,
        -1.8417, -1.4754, -0.4639, -0.4003,  0.5331,  0.9504,  0.9232, -0.7498,
        -0.5639,  1.2416, -1.0759,  0.0674,  0.7166,  1.7287, -0.8949, -0.6154,
        -0.5099,  0.5730, -1.1699, -2.1450,  1.0197,  0.4379,  0.6243, -1.2665,
        -0.4252, -2.2007,  1.3297, -1.5249, -1.0728,  0.7851,  0.4443, -0.6534,
         0.4012,  0.6339,  1.0410, -0.1818,  0.4915,  0.0586, -0.8483, -0.0779,
         0.3556, -1.3154,  0.3274, -0.56